In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
user_df = pd.read_csv('user_journeys_50000.csv')
ecommerce_df = pd.read_csv('ecommerce_funnel_50000.csv')
funnelsummary_df = pd.read_csv('funnel_summary_50000.csv')

In [4]:
user_df_clean = user_df[['session_id', 'page_sequence', 'sequence_length']]

cols_keep = ['session_id']  # always keep session_id
for col in funnelsummary_df.columns:
    if col not in ecommerce_df.columns:
        cols_keep.append(col)

funnelsummary_df_clean = funnelsummary_df[cols_keep]

# Merge ecommerce + user_df
merged_df = ecommerce_df.merge(user_df_clean, on='session_id', how='left')

# Merge ecommerce+user_df with funnel summary
merged_df = merged_df.merge(funnelsummary_df_clean, on='session_id', how='left')


In [5]:
merged_df.isnull().sum()

session_id                      0
user_id                         0
timestamp                       0
device_type                     0
operating_system                0
browser                         0
country                         0
traffic_source                  0
campaign_id                     0
user_type                       0
home_page_visit                 0
product_page_visit              0
cart_add                        0
checkout_start                  0
purchase_complete               0
cart_size                       0
cart_value                      0
page_views                      0
session_duration_seconds        0
time_on_product_seconds         0
products_viewed                 0
product_categories              0
bounced                         0
revenue                         0
discount_applied                0
shipping_cost                   0
exit_point                      0
abandonment_reason          48992
date                            0
hour_of_day   

In [93]:
merged_df.head()

,session_id,user_id,timestamp,device_type,operating_system,browser,country,traffic_source,campaign_id,user_type,...,day_of_week,month,funnel_stage,conversion_value_segment,session_type,device_category,region,traffic_quality,page_sequence,sequence_length
0,SESS_0000001,USER_816407,2024-02-04 07:42:30.964157,Mobile,Android,Chrome,India,Social Media,None,New,...,Sunday,February,Product,None,Browsing,Mobile,Asia,Low Quality,Home → Product List → Product Detail → Product...,12
1,SESS_0000002,USER_105013,2024-02-24 04:03:26.373116,Mobile,Android,Chrome,Australia,Organic Search,None,New,...,Saturday,February,Home,None,Bounced,Mobile,Oceania,High Quality,Home → Exit,2
2,SESS_0000003,USER_157375,2024-01-10 10:20:36.945401,Mobile,iOS,Chrome,UK,Paid Search,CAMP_4721,Returning,...,Wednesday,January,Home,None,Bounced,Mobile,Europe,Medium Quality,Home → Exit,2
3,SESS_0000004,USER_681115,2024-02-25 01:26:51.331315,Mobile,Android,Samsung Internet,France,Display Ads,CAMP_5789,Returning,...,Sunday,February,Purchase,Low (<$50),Converting,Mobile,Europe,Low Quality,Home → Product List → Product Detail → Product...,16
4,SESS_0000005,USER_728877,2024-01-07 01:34:25.438515,Mobile,Android,Safari,US,Email,CAMP_2892,New,...,Sunday,January,Home,None,Bounced,Mobile,North America,High Quality,Home → Exit,2


In [96]:
merged_df.session_id.nunique()


50000

In [97]:
merged_df.user_id.nunique()


48620

In [98]:
merged_df.revenue.sum()


933844.3657632761

In [99]:
#Conversion Rate
merged_df.purchase_complete.mean() * 100

8.744

In [100]:
#Bounce Rate
merged_df.exit_point.eq("Home Page").mean() * 100

59.24

In [101]:
merged_df[['home_page_visit','product_page_visit','cart_add','checkout_start','purchase_complete']].sum()


home_page_visit       50000
product_page_visit    20380
cart_add               7667
checkout_start         5380
purchase_complete      4372
dtype: int64

In [102]:
{
 "home_to_product_drop": 1 - (merged_df.product_page_visit.sum() / merged_df.home_page_visit.sum()),
 "product_to_cart_drop": 1 - (merged_df.cart_add.sum() / merged_df.product_page_visit.sum()),
 "cart_to_checkout_drop": 1 - (merged_df.checkout_start.sum() / merged_df.cart_add.sum()),
 "checkout_to_purchase_drop": 1 - (merged_df.purchase_complete.sum() / merged_df.checkout_start.sum())
}


{'home_to_product_drop': 0.5924,
 'product_to_cart_drop': 0.6237978410206084,
 'cart_to_checkout_drop': 0.29829137863571153,
 'checkout_to_purchase_drop': 0.18736059479553901}

In [103]:
#Cart Abandonment Rate
1 - (merged_df.purchase_complete.sum() / merged_df.cart_add.sum())


0.4297639233076823

In [104]:
merged_df.groupby("traffic_source").revenue.sum().sort_values(ascending=False)


traffic_source
Organic Search    272767.413357
Direct            257290.501241
Email             129557.668572
Paid Search        85761.114243
Social Media       78633.493237
Referral           54770.648208
Display Ads        31225.337799
Affiliate          23838.189106
Name: revenue, dtype: float64

In [105]:
merged_df.groupby("region").revenue.sum().sort_values(ascending=False)


region
North America    470358.370977
Europe           301242.651368
Asia              80006.612247
Oceania           59188.076556
South America     23048.654615
Name: revenue, dtype: float64

In [106]:
merged_df.groupby("device_category").revenue.sum()


device_category
Desktop/Tablet    632148.025613
Mobile            301696.340150
Name: revenue, dtype: float64

In [107]:
merged_df.groupby("user_type").revenue.sum()


user_type
Loyal         89169.452085
New          483528.682487
Returning    361146.231191
Name: revenue, dtype: float64

In [108]:
merged_df.groupby("user_type").purchase_complete.mean() * 100


user_type
Loyal        7.951375
New          8.604626
Returning    9.186396
Name: purchase_complete, dtype: float64

In [109]:
merged_df.conversion_value_segment.value_counts()


None                 45628
High (>$150)          2067
Medium ($50-$150)     2035
Low (<$50)             270
Name: conversion_value_segment, dtype: int64

In [110]:
merged_df.traffic_source.value_counts()


Organic Search    14088
Direct            12004
Social Media       6056
Email              5484
Paid Search        4540
Referral           3890
Display Ads        2460
Affiliate          1478
Name: traffic_source, dtype: int64

In [111]:
merged_df.device_category.value_counts()


Mobile            28809
Desktop/Tablet    21191
Name: device_category, dtype: int64

In [112]:
merged_df.groupby("operating_system").apply(lambda x: x.exit_point.eq("Home Page").mean() * 100)


operating_system
Android     62.056190
ChromeOS    54.755784
Linux       56.778523
Other       60.450161
Windows     54.875788
iOS         61.537574
macOS       55.622442
dtype: float64

In [113]:
merged_df.query("device_category=='Desktop/Tablet' and exit_point=='Home Page'").shape[0]


11755

In [114]:
merged_df.exit_point.eq("Product Page").sum()


12713

In [115]:
merged_df.exit_point.value_counts()


Home Page            29620
Product Page         12713
Purchase Complete     4372
Cart Page             2287
Checkout Page         1008
Name: exit_point, dtype: int64

In [117]:
merged_df.groupby("exit_point").products_viewed.mean().sort_values(ascending=False)


exit_point
Product Page         11.970581
Cart Page            11.966331
Purchase Complete    11.947164
Checkout Page        11.915675
Home Page             0.000000
Name: products_viewed, dtype: float64

In [118]:
merged_df.groupby("exit_point").time_on_product_seconds.mean().sort_values(ascending=False)


exit_point
Purchase Complete    64.301745
Checkout Page        63.700792
Product Page         63.121269
Cart Page            60.935510
Home Page             0.000000
Name: time_on_product_seconds, dtype: float64

In [119]:
merged_df.groupby("traffic_quality").purchase_complete.mean() * 100


traffic_quality
High Quality      9.782746
Low Quality       6.303782
Medium Quality    7.746145
Name: purchase_complete, dtype: float64

In [120]:
merged_df.discount_applied.sum()


1342

In [121]:
merged_df.query("user_type=='Returning' and discount_applied==True").shape[0]


482

In [122]:
merged_df.exit_point.eq("Checkout Page").sum()


1008

In [123]:
merged_df.groupby("device_category").purchase_complete.mean() * 100


device_category
Desktop/Tablet    10.348733
Mobile             7.563609
Name: purchase_complete, dtype: float64

In [124]:
merged_df.groupby("region").purchase_complete.mean() * 100


region
Asia              7.388787
Europe            8.955876
North America     9.153548
Oceania          10.061100
South America     6.539599
Name: purchase_complete, dtype: float64

In [125]:
merged_df.groupby("hour_of_day").session_id.count()


hour_of_day
0     2055
1     2052
2     2074
3     2059
4     2056
5     2166
6     2106
7     2076
8     2080
9     2193
10    2049
11    2036
12    2124
13    2095
14    2101
15    2121
16    2082
17    2088
18    2023
19    2041
20    2138
21    2071
22    2081
23    2033
Name: session_id, dtype: int64

In [126]:
merged_df.groupby("day_of_week").session_id.count()


day_of_week
Friday       7232
Monday       6909
Saturday     7212
Sunday       6912
Thursday     7345
Tuesday      7197
Wednesday    7193
Name: session_id, dtype: int64

In [127]:
merged_df.groupby("month").revenue.sum()


month
February    296260.242482
January     315802.508037
March       321781.615245
Name: revenue, dtype: float64

In [128]:
merged_df['region'].value_counts()

North America    21915
Europe           16023
Asia              6564
South America     3043
Oceania           2455
Name: region, dtype: int64

In [129]:
merged_df['device_category'].value_counts()

Mobile            28809
Desktop/Tablet    21191
Name: device_category, dtype: int64

In [130]:
merged_df['session_type'].value_counts()

Bounced        29620
Browsing       12713
Converting      4372
Engaged         2287
High Intent     1008
Name: session_type, dtype: int64

In [131]:
merged_df['conversion_value_segment'].value_counts()

None                 45628
High (>$150)          2067
Medium ($50-$150)     2035
Low (<$50)             270
Name: conversion_value_segment, dtype: int64

In [132]:
merged_df['funnel_stage'].value_counts()

Home        29620
Product     12713
Purchase     4372
Cart         2287
Checkout     1008
Name: funnel_stage, dtype: int64

In [133]:
merged_df['exit_point'].value_counts()

Home Page            29620
Product Page         12713
Purchase Complete     4372
Cart Page             2287
Checkout Page         1008
Name: exit_point, dtype: int64

In [134]:
merged_df['product_categories'].value_counts()

None                                           29620
Electronics                                      566
Home & Garden                                    524
Office Supplies                                  522
Sports                                           518
                                               ...  
Home & Garden, Automotive, Toys, Books             1
Books, Toys, Sports, Beauty                        1
Clothing, Automotive, Home & Garden, Sports        1
Toys, Sports, Beauty, Clothing                     1
Automotive, Toys, Sports, Clothing                 1
Name: product_categories, Length: 4041, dtype: int64

In [135]:
merged_df['traffic_source'].value_counts()

Organic Search    14088
Direct            12004
Social Media       6056
Email              5484
Paid Search        4540
Referral           3890
Display Ads        2460
Affiliate          1478
Name: traffic_source, dtype: int64

In [136]:
merged_df['date'].min()

'2024-01-01'

In [137]:
merged_df['date'].max()


'2024-03-31'

In [138]:
merged_df['user_type'].value_counts()

New          27369
Returning    17613
Loyal         5018
Name: user_type, dtype: int64

In [139]:
merged_df.groupby('traffic_source')['revenue'].sum().sort_values(ascending=False)

traffic_source
Organic Search    272767.413357
Direct            257290.501241
Email             129557.668572
Paid Search        85761.114243
Social Media       78633.493237
Referral           54770.648208
Display Ads        31225.337799
Affiliate          23838.189106
Name: revenue, dtype: float64